<a href="https://colab.research.google.com/github/buketugurlu/dentaLyze/blob/main/yolov11_deneme5v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Ultralytics (YOLO) paketini yükle
!pip install ultralytics

# Albumentations ve OpenCV paketlerini yükle
!pip install albumentations opencv-python-headless

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 896.9/896.9 kB 16.4 MB/s eta 0:00:00


In [2]:
pip install -U albumentations

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.0/66.0 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.9/227.9 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 632.7/632.7 kB 26.3 MB/s eta 0:00:00
  Attempting uninstall: albucore
    Found existing installation: albucore 0.0.19
    Uninstalling albucore-0.0.19:
      Successfully uninstalled albucore-0.0.19
  Attempting uninstall: albumentations
    Found existing installation: albumentations 1.4.20
    Uninstalling albumentations-1.4.20:
      Successfully uninstalled albumentations-1.4.20


In [3]:
!rm -rf /content/drive
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [4]:
from ultralytics import YOLO
import os
import yaml
import cv2
from albumentations import Compose, HorizontalFlip, ShiftScaleRotate, Affine

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [5]:
# 1. Dataset YAML dosyasını oluşturma
dataset_yaml_path = "/content/drive/MyDrive/Dental OPG XRAY Dataset/dataset.yaml"
yaml_dir = os.path.dirname(dataset_yaml_path)

# Dizini kontrol et ve gerekirse oluştur
os.makedirs(yaml_dir, exist_ok=True)
yaml_content = {
    "path": "/content/drive/MyDrive/Dental OPG XRAY Dataset/Augmented_Data",
    "train": "train/images",
    "val": "valid/images",
    "test": "test/images",
    "nc": 6,
    "names": ['Healthy Teeth', 'Caries', 'Impacted Teeth', 'BDC/BDR', 'Infection', 'Fractured Teeth']
}

In [6]:
# YAML dosyasını kaydet
with open(dataset_yaml_path, "w") as yaml_file:
    yaml.dump(yaml_content, yaml_file)

print(f"Dataset YAML dosyası oluşturuldu: {dataset_yaml_path}")

Dataset YAML dosyası oluşturuldu: /content/drive/MyDrive/Dental OPG XRAY Dataset/dataset.yaml


In [7]:
# CLAHE İşleme
def apply_clahe(input_dir, output_dir):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    for img_file in os.listdir(input_dir):
        img_path = os.path.join(input_dir, img_file)
        output_path = os.path.join(output_dir, img_file)
        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
        if img is None:
            continue
        clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
        enhanced_img = clahe.apply(img)
        cv2.imwrite(output_path, enhanced_img)

In [8]:
# Augmentasyon
def augment_image(image_path, save_path):
    image = cv2.imread(image_path)
    transform = Compose([
        HorizontalFlip(p=0.5),
        ShiftScaleRotate(shift_limit=0.0625, scale_limit=0.2, rotate_limit=15, p=0.7),
        Affine(shear=15, p=0.5),
    ])
    augmented = transform(image=image)
    cv2.imwrite(save_path, augmented["image"])

In [12]:
# Modeli Eğitme
def train_model():
    model = YOLO('/content/drive/MyDrive/models/yolo11x.pt')
    model.train(
        data=dataset_yaml_path,
        epochs=100,
        batch=16,
        imgsz=640,
        name='dental_detection',
        plots=True
    )

In [13]:
# Tahmin
def predict_images():
    model = YOLO('runs/train/dental_detection/weights/best.pt')
    results = model.predict(
        source='/content/drive/MyDrive/Dental OPG XRAY Dataset/Augmented_Data/test/images',
        conf=0.25,
        save=True,
        save_txt=True,
        save_conf=True
    )
    return results

In [14]:
# Ana Fonksiyon
def main():
    try:
        input_dir = "/content/drive/MyDrive/Dental OPG XRAY Dataset/Augmented_Data/train/images"
        clahe_dir = "/content/drive/MyDrive/Dental OPG XRAY Dataset/Augmented_Data/train/images_clahe"
        augmented_dir = "/content/drive/MyDrive/Dental OPG XRAY Dataset/Augmented_Data/train/images_augmented"

        print("CLAHE uygulanıyor...")
        apply_clahe(input_dir, clahe_dir)
        print("Augmentasyon uygulanıyor...")
        os.makedirs(augmented_dir, exist_ok=True)
        for img_file in os.listdir(clahe_dir):
            augment_image(os.path.join(clahe_dir, img_file), os.path.join(augmented_dir, img_file))

        print("Eğitim başlatılıyor...")
        train_model()

        print("Tahmin başlatılıyor...")
        results = predict_images()
        print("Tahmin tamamlandı.")
    except Exception as e:
        print(f"Hata oluştu: {e}")

if __name__ == "__main__":
    main()

CLAHE uygulanıyor...
Augmentasyon uygulanıyor...
Eğitim başlatılıyor...
Ultralytics 8.3.39 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (NVIDIA A100-SXM4-40GB, 40514MiB)
engine/trainer: task=detect, mode=train, model=/content/drive/MyDrive/models/yolo11x.pt, data=/content/drive/MyDrive/Dental OPG XRAY Dataset/dataset.yaml, epochs=100, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=dental_detection, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=Non

100%|██████████| 755k/755k [00:00<00:00, 15.4MB/s]


Overriding model.yaml nc=80 with nc=6

                   from  n    params  module                                       arguments                     
  0                  -1  1      2784  ultralytics.nn.modules.conv.Conv             [3, 96, 3, 2]                 
  1                  -1  1    166272  ultralytics.nn.modules.conv.Conv             [96, 192, 3, 2]               
  2                  -1  2    389760  ultralytics.nn.modules.block.C3k2            [192, 384, 2, True, 0.25]     
  3                  -1  1   1327872  ultralytics.nn.modules.conv.Conv             [384, 384, 3, 2]              
  4                  -1  2   1553664  ultralytics.nn.modules.block.C3k2            [384, 768, 2, True, 0.25]     
  5                  -1  1   5309952  ultralytics.nn.modules.conv.Conv             [768, 768, 3, 2]              
  6                  -1  2   5022720  ultralytics.nn.modules.block.C3k2            [768, 768, 2, True]           
  7                  -1  1   5309952  ultralytics

100%|██████████| 5.35M/5.35M [00:00<00:00, 67.4MB/s]


AMP: checks passed ✅


...

    100/100      16.5G     0.8566     0.7969      1.078          3        640: 100%|██████████| 94/94 [00:23<00:00,  4.01it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 12/12 [00:01<00:00,  6.36it/s]

                   all        383        546      0.103      0.474      0.098     0.0624



100 epochs completed in 0.771 hours.
Optimizer stripped from runs/detect/dental_detection/weights/last.pt, 114.4MB
Optimizer stripped from runs/detect/dental_detection/weights/best.pt, 114.4MB

Validating runs/detect/dental_detection/weights/best.pt...
Ultralytics 8.3.39 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (NVIDIA A100-SXM4-40GB, 40514MiB)
YOLO11x summary (fused): 464 layers, 56,833,954 parameters, 0 gradients, 194.4 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 12/12 [00:02<00:00,  4.72it/s]


                   all        383        546      0.104       0.52        0.1     0.0648
         Healthy Teeth         53         91      0.104      0.505     0.0739     0.0451
                Caries         14         23       0.17      0.609      0.132     0.0757
        Impacted Teeth         37         86      0.113        0.5      0.104     0.0645
               BDC/BDR          7          7      0.123      0.714      0.144      0.118
             Infection         21         36      0.055      0.444      0.101      0.054
       Fractured Teeth        100        303      0.057      0.347     0.0469     0.0313
Speed: 0.1ms preprocess, 2.3ms inference, 0.0ms loss, 1.5ms postprocess per image
Results saved to runs/detect/dental_detection
Tahmin başlatılıyor...
Hata oluştu: [Errno 2] No such file or directory: 'runs/train/dental_detection/weights/best.pt'
